In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
breweries= pd.read_csv('data/breweries.csv')

In [4]:
url_list=[ x for x in breweries['Untappd']]
brewery_list=[ x for x in breweries['Brewery']]

In [5]:
#url_list

In [6]:
#breweries

In [7]:
brewery_id = []
x=0
for brewery in breweries['Brewery']:
    brewery_id += 5 * [x]
    x +=1
brewery_id.pop(263)

52

In [8]:
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]

for url in breweries['Untappd']:

    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    beer = soup.find('div', class_='beer-container')
    beer_name_search = beer.find_all('p', class_='name')[:5]
    beer_type_search = beer.find_all('p', class_='style')[:5]

    beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
    beer_abv_search2 = beer.find_all('p', class_='abv')[:5]

    beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
    beer_ibu_search2 = beer.find_all('p', class_='ibu')[:5]

    beer_rating_search = beer.find_all('span', class_='num')[:5]

    beer_popular = beer.find_all('div', class_='details-item raters')[:5]
    beer_popular2 = beer.find_all('p', class_='raters')[:5]

    for paragraph in beer_name_search:
        if paragraph.a:
            beer_name.append(paragraph.a.text)

    for paragraph in beer_type_search:
        if paragraph.text:
            beer_type.append(paragraph.text)

    for paragraph in beer_abv_search:
        if paragraph.text:
            beer_abv.append(paragraph.text)
    for paragraph in beer_abv_search2:
        if paragraph.text:
            beer_abv.append(paragraph.text)

    for paragraph in beer_ibu_search:
        if paragraph.text:
            beer_ibu.append(paragraph.text)
    for paragraph in beer_ibu_search2:
        if paragraph.text:
            beer_ibu.append(paragraph.text)

    for paragraph in beer_rating_search:
        if paragraph.text:
            beer_rating.append(paragraph.text)

    for paragraph in beer_popular:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)
    for paragraph in beer_popular2:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)

In [9]:
print(len(beer_name))
print(len(beer_type))
print(len(beer_abv))
print(len(beer_ibu))
print(len(beer_rating))
print(len(beer_number_rating))

279
279
279
279
279
279


In [10]:
#columns for schema

#brewery_id
#name
#type
#abv
#ibu
#rating
#rating_count
beer_df = pd.DataFrame({'Brewery_ID':brewery_id,
                       'Name':beer_name,
                       'Type': beer_type,
                       'ABV': beer_abv,
                       'IBU':beer_ibu,
                       'Rating':beer_rating,
                       'Rating_Count':beer_number_rating})

In [11]:
beer_df.to_csv('data/beer_list_dirty.csv')
beer_df.head(10)

,Brewery_ID,Name,Type,ABV,IBU,Rating,Rating_Count
0,0,Long Walk,IPA - American,\n7% ABV,\n57 IBU,(3.61),"\n5,723 Ratings"
1,0,Kung Fu Robot,IPA - American,\n7% ABV,\n43 IBU,(3.65),"\n4,707 Ratings"
2,0,Vitality Light Ale,Blonde Ale,\n5% ABV,\n17 IBU,(3.46),"\n2,687 Ratings"
3,0,Sun Eater,Gruit / Ancient Herbed Ale,\n4.8% ABV,\nN/A IBU,(3.48),"\n2,579 Ratings"
4,0,Highland Scoundrel,Scotch Ale / Wee Heavy,\n10.6% ABV,\n20 IBU,(3.84),"\n2,404 Ratings"
5,1,Naked Nun,Wheat Beer - Witbier,\n5.8% ABV,\nN/A IBU,(3.6),"\n10,933 Ratings"
6,1,Tripel B,Belgian Tripel,\n9.3% ABV,\nN/A IBU,(3.87),"\n10,253 Ratings"
7,1,Philosophizer,Farmhouse Ale - Saison,\n7.8% ABV,\n36 IBU,(3.8),"\n9,745 Ratings"
8,1,Hibiscus Saison,Farmhouse Ale - Saison,\n5.6% ABV,\n33 IBU,(3.65),"\n7,303 Ratings"
9,1,Flyin' Monks,Belgian Quadrupel,\n10.9% ABV,\nN/A IBU,(3.95),"\n6,307 Ratings"


In [12]:
#clean ABV
beer_df['ABV']=beer_df['ABV'].str.replace('% ABV','')
beer_df['ABV']=beer_df['ABV'].str.replace('\n','')
#pd.to_numeric()

In [13]:
#convert to number, errors = coerce replace other values as NaN
beer_df['ABV']=pd.to_numeric(beer_df['ABV'],errors='coerce')

In [14]:
#clean IBU column
beer_df['IBU']=beer_df['IBU'].str.replace(' IBU','')
beer_df['IBU']=beer_df['IBU'].str.replace('\n','')
beer_df['IBU']=pd.to_numeric(beer_df['IBU'],errors='coerce')

In [15]:
#clean Rating Column
beer_df['Rating']=beer_df['Rating'].str.replace('(','')
beer_df['Rating']=beer_df['Rating'].str.replace(')','')
beer_df['Rating']=pd.to_numeric(beer_df['Rating'],errors='coerce')

In [16]:
#clean Rating_Count Column
beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace('\n','')
beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace(',','')
beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace(' Ratings','')
beer_df['Rating_Count']=pd.to_numeric(beer_df['Rating_Count'],errors='coerce')

In [17]:
beer_df_clean = beer_df

In [18]:
beer_df_clean.to_csv('data/beer_list_clean.csv')

In [19]:
beer_df_clean.tail(20)

,Brewery_ID,Name,Type,ABV,IBU,Rating,Rating_Count
259,51,ProtoModern IPA,IPA - American,7.1,75.0,3.73,3053
260,52,Panzer IPA,IPA - American,NaN,NaN,3.52,13
261,52,Fat Bottom Blonde,Blonde Ale,NaN,NaN,NaN,6
262,52,Route Irish Red,Red Ale - Irish,NaN,NaN,NaN,3
263,52,Hill Country Hefeweizen,Hefeweizen,NaN,NaN,NaN,4
264,53,Funky Monkey IPA,IPA - American,7.7,54.0,3.75,650
265,53,Moxey Monkey,IPA - New England,5.5,54.0,3.72,487
266,53,Monkey Wrench Session IPA,IPA - Session / India Session Ale,5.5,54.0,3.55,446
267,53,Monkey Tail,Pale Ale - American,5.6,41.0,3.52,412
268,53,Punk Monkey Porter,Porter - American,5.6,30.0,3.64,404


In [ ]:
#debug 
test=[]
for url in breweries['Untappd']:
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    beer = soup.find('div', class_='beer-container')
    if beer is None:
        test.append('error')
    else:
        test.append('success')

In [ ]:
#test

In [ ]:
beer_len=[]

for url in breweries['Untappd']:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    beer = soup.find('div', class_='beer-container')

    beer_name_search = beer.find_all('p', class_='name',limit=5)

    beer_len.append(len(beer_name_search))

In [ ]:
#beer_len

In [ ]:
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]


url =  breweries['Untappd'][33]
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

beer = soup.find('div', class_='beer-container')

beer_name_search = beer.find_all('p', class_='name',limit=5)

eer_type_search = beer.find_all('p', class_='style',limit=5)

beer_abv_search = beer.find_all('div', class_='details-item abv',limit=5)
beer_abv_search2 = beer.find_all('p', class_='abv',limit=5)[:5]

beer_ibu_search = beer.find_all('div', class_='details-item ibu',limit=5)
beer_ibu_search2 = beer.find_all('p', class_='ibu',limit=5)[:5]

beer_rating_search = beer.find_all('span', class_='num',limit=5)[:5]

beer_popular = beer.find_all('div', class_='details-item raters',limit=5)
beer_popular2 = beer.find_all('p', class_='raters',limit=5)

for paragraph in beer_name_search:
    if paragraph.a:
        beer_name.append(paragraph.a.text)

for paragraph in beer_type_search:
    if paragraph.text:
        beer_type.append(paragraph.text)

for paragraph in beer_abv_search:
    if paragraph.text:
        beer_abv.append(paragraph.text)
for paragraph in beer_abv_search2:
    if paragraph.text:
        beer_abv.append(paragraph.text)

for paragraph in beer_ibu_search:
    if paragraph.text:
        beer_ibu.append(paragraph.text)
for paragraph in beer_ibu_search2:
    if paragraph.text:
        beer_ibu.append(paragraph.text)

for paragraph in beer_rating_search:
    if paragraph.text:
        beer_rating.append(paragraph.text)

for paragraph in beer_popular:
    if paragraph.text:
        beer_number_rating.append(paragraph.text)
for paragraph in beer_popular2:
    if paragraph.text:
        beer_number_rating.append(paragraph.text)

In [ ]:
print(len(beer_name))
print(len(beer_type))
print(len(beer_abv))
print(len(beer_ibu))
print(len(beer_rating))
print(len(beer_number_rating))